# Tutorial 3.6: Pandas Data Merging
Python for Data Analytics | Module 3  
Professor James Ng

In [ ]:
# SETUP: DO NOT CHANGE
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

## Introduction

In this tutorial, we will be exploring another way to combine datasets through the `pd.merge()` function.

Those who have a background in databases will find a significant amount of overlap between your SQL work and the `pd.merge()` function. But, if you don't have this background, not to worry, we will take it one step at a time.

In [ ]:
# For this tutorial, we will be combining data from both the 
# college load defaults and college scorecard data sets
# so we will need to load them both.
college_loan_defaults = pd.read_csv(
    'datasets/college-loan-default-rates.csv')
college_loan_defaults.head()

In [ ]:
college_scorecard = pd.read_csv(
    'datasets/college-scorecard-data-scrubbed.csv', 
    encoding='latin-1')
college_scorecard.head()

## The Difference between `pd.concat()` & `pd.merge()`
The essential difference between concatenation and merging is that the latter requires the existence of one or more shared columns (or indices) between the two dataframes.

Concatenation has no such requirement. It will simply slap together whatever you give it and fill `NaN` values into spots where there are column/row mismatches.

Let's demonstrate this quickly:

In [ ]:
# Here are the first 5 rows and 2 random columns of college_loan_defaults
college_loan_defaults.iloc[:5, 2:4]

In [ ]:
# Here are the same number of elements from college_scorecard
college_scorecard.iloc[:5, 0:2]

As you can see, there are no matching values between these dataframes.  That does not prevent us from concatenating them:

In [ ]:
# Concatenating Rows
pd.concat([college_loan_defaults.iloc[:5, 2:4], 
           college_scorecard.iloc[:5, 0:2]])

In [ ]:
# Concatenating Columns
pd.concat(
    [college_loan_defaults.iloc[:5, 2:4],college_scorecard.iloc[:5, 0:2]], axis=1)

A merge operation could not be performed between these two *DataFrame* objects because they don't have any shared key values to cross-reference.

## The 3 Categories of Merges/Joins
There are 3 different categories of merges/joins which are defined by the characteristics of the shared columns/indices:
* One-to-One: Each shared key value exists only once in both dataframes.
* One-to-Many: A given shared key value exists once in first dataframe, but 1 or more times in the second dateframe.
* Many-to-Many: A given shared key value exists 1 or more times in both dataframes.

Let's provide an example of each type of merge from our datasets.

### One-to-One Join

Let's say that instead of receiving the college scorecard dataset as one whole unit, we had received it in two parts. One dataset had some cost information, and the other had information about median student earnings and debt after attendence.

I'll generate these *DataFrame* objects for you:

In [ ]:
college_costs = college_scorecard[
    ['OPEID', 'average_net_price_public', 'average_net_price_private']][5:10]
college_costs.head()

In [ ]:
college_outcomes = college_scorecard[
    ['OPEID', 'median_student_earnings', 'median_student_debt']][7:12]
college_outcomes

The **`OPEID`** field in the college_scorecard dataset is a unique field, which means that each value is guaranteed to only appear once. 

Because of this, if we merge the two dataframes it will be a 1-1 join.

In [ ]:
pd.merge(college_costs, college_outcomes)

Great. Here's what *pandas* did:
1. Identified the matching column(s) between the two dataframes: `OPEID`.
1. Found matching `OPEID` values between the two dataframes.
1. Merged the columns of matching `OPEID` values together.
1. **Important**: Generated a new numeric index.

<div class="alert alert-block alert-info">
<p>
    In our discussion, we will reference the columns that <em>pandas</em> is using to find matches between dataframes as the "join column(s)" or "merge column(s)".
</p>
</div>

#### Controlling the Join Type with the `how` Parameter
Did you notice that some of the records from each of the original dataframes didn't make it into the merge product?

This is because the type of merge that was applied to the dataframes was called an **inner join** - which is the default.

The are actually 4 types of joins that you can use in a merge operation:
* **Inner Join**: To be included in the output dataframe, the join column(s) value must exist in both original dataframes. 
    * This is why some of the records didn't get included in the output, because they didn't have a corresponding join column(s) values in the other dataframe.
* **Outer Join**: All records from both dataframes are included in the output. *pandas* simply fills in `NaN` where there are no corresponding join column(s) value.
* **Left Join**: All rows from the first (left) dataframe will be included in the output dataframe, regardless of whether there are matching join column value(s) in the second (right) dataframe.
* **Right Join**: All rows from the second (right) dataframe will be included in the output dataframe, regardless of whether there are matching join column value(s) in the left (first) dataframe.

Let's go ahead and try all these different types of joins to see how our output changes.

In [ ]:
# Outer Join
# All records from both dataframes are included.
# NaN is inserted into missing grid points.
pd.merge(college_costs, college_outcomes, how="outer")

In [ ]:
# Left Join
# All records from the first/left (college_costs) dataframe are included.
# NaN is inserted into missing grid points.
pd.merge(college_costs, college_outcomes, how="left")

In [ ]:
# Right Join
# All records from the second/right (college_outcomes) 
# dataframe are included.
# NaN is inserted into missing grid points.
pd.merge(college_costs, college_outcomes, how="right")

### One-to-Many Join

Notice that the `college_scorecard` data set has three ID columns: **UNITID**, **OPEID** and **OPEID6**.

The reason for this is that universities with multiple locations have multiple records in the scorecard data, but a single `OPEID6` value for the organization as a whole.

Let's take a closer look at the duplication.


In [ ]:
whichdup = college_scorecard.duplicated(subset=['OPEID6'], keep=False)
college_scorecard[whichdup]

# Sort this by OPEID and OPEID6 for a better view
college_scorecard[whichdup==True].sort_values(by = ['OPEID6'])

This sets up a one-to-many join/merge opportunity for us between the Scorecard data set and the College Loan Defaults data set, which only has 1 instance of each `OPEID6` value.

In [ ]:
# First let's grab the records from `college_scorecard`
# for a multi-campus university - Fortis College - that will have 
# a repeated `OPEID6` value.

# We will just grab a couple of columns for this university to
# keep things simple.
fortis_college = college_scorecard[
    college_scorecard['OPEID6'] == 23410][['OPEID', 'OPEID6', 'UNITID', 'institution_name', 'degree_seeking_undergrads', 'students_with_federal_loans']]
fortis_college

In [ ]:
# Now let's grab the corresponding fragement of 
# the default rates for Fortis College's OPEID
fortis_college_default_rate = college_loan_defaults[
    college_loan_defaults['opeid'] == 23410][['opeid', 'year_1_default_rate']]
fortis_college_default_rate

Now let's go ahead and try to perform the merge:

In [ ]:
pd.merge(fortis_college, fortis_college_default_rate)

#### Specifying the Join Columns
Well...that isn't want we wanted. :(

Thankfully though, the error message is pretty self-explanatory. *pandas* thinks there are no common columns to merge on.

The reason for this is that the common values are held in columns with slightly different names. We have to explain to *pandas* what to do when this happens by specifying the names of the columns to join on.

In [ ]:
# Use the left_on and right_on parameters to specify the
# name(s) of the join column(s) in the first(left)
# and second (right) dataframes.
pd.merge(
    fortis_college, 
    fortis_college_default_rate,
    left_on='OPEID6',
    right_on='opeid')

<div class="alert alert-block alert-info">
<h5>There can be more than 1 join column</h5>
<p>
In this example, we have specified only one join column. But you can specify multiple columns if you so desire. Just pass them as a list to the `left_on` and `right_on` parameters.
</p>
</div>

**Ah, much better.** You can see (you might have to expand your screen a bit) how *pandas* took the **one** default rate record and applied it to **many** corresponding scorecard records for Fortis College.

You could then proceed from this to perform calculations like approximating the number of students in default for each campus.

### Many-to-Many Join

It is a little bit of a stretch to come up with a good many-to-many join example from our two datasets, so let's create a couple of imaginary ones that will illustrate this merge/join category well.

We will have two dataframes: 
* The first will list the members of a work group and their favorite restaurants. 
* The second will contain some of the foods served by each restaurant. 

The key here is that a given restaurant has the potential of appearing more than once in both data sets. This sort of situation is what lies behind a "many-to-many" merge.  

It is easier to see than to explain so let's create our datasets and perform the merge.

In [ ]:
# Team Members Favorite Restaurants
team_member_favorite_restaurants = pd.DataFrame(
    {
        'member': ['Mike', 'Kim', 'Roger', 'Sam', 'Sonia'],
     'restaurant': ['In-N-Out', 'Chipotle', 'Chick-Fil-A', 'Chick-Fil-A', 'In-N-Out']
    }
)
team_member_favorite_restaurants

In [ ]:
# Restaurant Items
restaurant_items = pd.DataFrame(
    {
        'item': [
        'Burgers', 'Fries', 'Shakes', 
        'Tacos', 'Burritos', 'Chips',
        'Chicken Sandwich', 'Fries', 'Salads'
        ]
        ,
        'restaurant': [
        'In-N-Out', 'In-N-Out', 'In-N-Out', 
        'Chipotle', 'Chipotle', 'Chipotle',
        'Chick-Fil-A', 'Chick-Fil-A', 'Chick-Fil-A'
        ]
    }
)
restaurant_items

You can see that in our first dataframe, In-N-Out and Chick-Fil-A both appear twice, and all restaurants appear three times in our second dataframe.  Let's perform our merge:

In [ ]:
# Notice how we're using a combination of columns
# and indexes for the "join columns"
pd.merge(team_member_favorite_restaurants, restaurant_items, 
         on='restaurant')

You can see here that many-to-many joins have a multiplicative effect. 

In practice, many-to-many merge is relatively rare compared to the others, but it is important to understand and can yield some interesting analysis possibilities... such as determining which food item is most likely to be popular with your team.  In this case, the answer is fries.

## EXERCISE 1

Merge the two datasets below into a single dataset of annual GDP and happiness in the USA. The first dataset contains GDP and the second dataset contains happiness data. The years in each dataset are not the same. Compare the results of the default inner join, left, right and outer.

In [ ]:
dfhappy = pd.read_csv('datasets/happy_annual.csv')
dfgdp = pd.read_csv('datasets/gdp_annual.csv')

In [ ]:
pd.merge(dfhappy, dfgdp)

In [ ]:
pd.merge(dfhappy, dfgdp, how='left')

In [ ]:
pd.merge(dfhappy, dfgdp, how='right')

In [ ]:
pd.merge(dfhappy, dfgdp, how='outer')

#### To reproduce the figure below, how should you join the two datasets?

<img src="datasets/gdphappy0.png"></img>

In [ ]:
# Here's the code to generate the above image. 
# It assumes your merged data are in df_happy_gdp.
fig, ax1 = plt.subplots()
barhappy = ax1.plot(df_happy_gdp['year'], df_happy_gdp['happy'])
ax2 = ax1.twinx()
linegdp = ax2.plot(df_happy_gdp['year'], df_happy_gdp['gdppc'], linestyle='dashed', color='green')

# add legend
barhappy, label1 = ax1.get_legend_handles_labels()
linegdp, label2 = ax2.get_legend_handles_labels()
ax2.legend(barhappy + linegdp, label1 + label2)

## EXERCISE 2
Merge the two datasets below into a single dataset containing all movies and their user ratings. The first dataset contains movie titles and the second dataset contains movie ratings. You'll have to figure out the column to join on, and how the join should be done (left, right, inner?).

In [ ]:
titles = pd.read_csv('datasets/movie_titles.csv', encoding='latin-1')
ratings = pd.read_csv('datasets/movie_ratings.csv')